In [1]:
from __future__ import print_function
from __future__ import print_function
from __future__ import division
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
import matplotlib.pyplot as plt
import csv
from torchvision import models
import time
import os
import copy
import cv2
import numpy as np
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import scipy.io
import numpy as np
from pathlib import Path
import glob

import torch.utils.data as data
import pandas as pd
from PIL import Image
import os.path

PyTorch Version:  2.1.2+cu121
Torchvision Version:  0.16.2+cu121


In [2]:
IMG_EXTENSIONS = [
   '.jpg', '.JPG', '.jpeg', '.JPEG',
   '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP','.mat',
]


def is_image_file(filename):
   return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def find_classes(dir):
   classes = os.listdir(dir)
   classes.sort()
   class_to_idx = {classes[i]: i for i in range(len(classes))}
   return classes, class_to_idx


def make_dataset(dir, class_to_idx):
   images = []
   for target in os.listdir(dir):
       d = os.path.join(dir, target)
       if not os.path.isdir(d):
           continue

       for filename in os.listdir(d):
           if is_image_file(filename):
               path = '{0}/{1}'.format(target, filename)
               #print(path)
               item = (path, class_to_idx[target])
               images.append(item)

   return images

def default_loader(path):
   return Image.open(path).convert('RGB')

def mat_loader(path):
   return scipy.io.loadmat(path)

In [3]:
classes1, class_to_idx1 = find_classes("/home/user1/icip/sim_Liu/train/")
       
imgs1 = make_dataset("/home/user1/icip/random_Liu/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/sim_Liu/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

print("="*100)

imgs1 = make_dataset("/home/user1/icip/Merged_Liu/test/", class_to_idx1)
print(class_to_idx1,len(imgs1))

{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_Note_8Pro': 7, 'D09_Samsung_Galaxy_J8_10G': 8, 'D10_Samsung_Galaxy_F41': 9, 'D11_OnePlus_8T': 10, 'D12_Vivo_Y02t': 11, 'D13_Oppo_A17k': 12, 'D14_Samsung_Galaxy_S20FE': 13, 'D15_Motorola_Motog60': 14, 'D16_Samsung_Galaxy_S21FE': 15, 'D17_Apple_Iphone_12': 16, 'D18_IQOO_Z3': 17, 'D19_IQOO_Z6_Lite': 18, 'D20_Motorola_MotoG73_5G': 19, 'D21_OnePlus_10Pro_5G': 20, 'D22_Poco_F5': 21, 'D23_Poco_F5_Pro_5G': 22, 'D24_Realme_8': 23, 'D25_Realme_X3_Superzoom': 24, 'D26_Redmi_9i_Sport': 25, 'D27_Redmi_Note10_Pro': 26, 'D28_Apple_Iphone_13': 27, 'D29_Apple_Iphone_15': 28, 'D30_Vivo_Y75': 29} 133120
{'D01_Samsung_Galaxy_S20Plus': 0, 'D02_Nothing_One': 1, 'D03_Samsung_Galaxy_A03': 2, 'D04_Samsung_Galaxy_M04': 3, 'D05_Vivo_V9_Pro': 4, 'D06_Apple_Iphone_12Mini': 5, 'D07_Apple_Iphone_11': 6, 'D08_Redmi_No

In [4]:
class ImageFolderLoader(data.Dataset):
   def __init__(self, root1,transform_1=None,
                target_transform=None,
                loader=default_loader):
       classes1, class_to_idx1 = find_classes(root1)
       
       imgs1 = make_dataset(root1, class_to_idx1)
      

       self.root1 = root1
       self.imgs1 = imgs1
       self.classes1 = classes1
       self.class_to_idx1 = class_to_idx1
       self.target_transform = target_transform
       self.loader = loader
       self.img_transform = transform_1
        
       
       

   def __getitem__(self, index):
    

       path1, target1 = self.imgs1[index]
       filename = Path(path1).stem 
       
    
       img1 = self.loader(os.path.join(self.root1, path1))  
       
       if self.img_transform is not None:
           img1 = self.img_transform(img1)
        
       if self.target_transform is not None:
           target1 = self.target_transform(target)
        
       target1 = torch.eye(30)[target1]      
            
       return img1,target1,filename

   def __len__(self):
       return len(self.imgs1)

In [5]:
data_transforms = transforms.Compose([
transforms.ToTensor()
])

batchsize=1

val_dataset_random = ImageFolderLoader(
        "/home/user1/icip/random_Liu/test/",
        transform_1=data_transforms
    )

test_loader_random = torch.utils.data.DataLoader(
        val_dataset_random, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_similar = ImageFolderLoader(
        "/home/user1/icip/sim_Liu/test/",
        transform_1=data_transforms
    )

test_loader_similar = torch.utils.data.DataLoader(
        val_dataset_similar, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

val_dataset_merged = ImageFolderLoader(
        "/home/user1/icip/Merged_Liu/test/",
        transform_1=data_transforms
    )

test_loader_merged = torch.utils.data.DataLoader(
        val_dataset_merged, batch_size=batchsize,
        shuffle=False, num_workers=4
  )

In [6]:
len(val_dataset_random), len(val_dataset_similar), len(val_dataset_merged)

(133120, 122880, 256000)

In [7]:
class Res2Net(nn.Module):

    def __init__(self):
       
        super(Res2Net, self).__init__()
        inplanes = 3
        planes = 3
        
        width = 16
        self.width_1 = 16
        scale = 4
        stride = 1
        
        self.conv1 = nn.Conv2d(inplanes, width*scale, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width*scale)
        self.relu1 = nn.ReLU(inplace=True)
       # self.relu = nn.ReLU(inplace=True)
        
        self.nums = scale
        
        self.conv11 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn11 = nn.BatchNorm2d(width)
        self.relu11 = nn.ReLU(inplace=True)
        self.conv12 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn12 = nn.BatchNorm2d(width)
        self.relu12 = nn.ReLU(inplace=True)
        
        
        self.conv21 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn21 = nn.BatchNorm2d(width)
        self.relu21 = nn.ReLU(inplace=True)
        self.conv22 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn22 = nn.BatchNorm2d(width)
        self.relu22 = nn.ReLU(inplace=True)
        
        self.conv31 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn31 = nn.BatchNorm2d(width)
        self.relu31 = nn.ReLU(inplace=True)
        self.conv32 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn32 = nn.BatchNorm2d(width)
        self.relu32 = nn.ReLU(inplace=True)
        
        self.conv41 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn41 = nn.BatchNorm2d(width)
        self.relu41 = nn.ReLU(inplace=True)
        self.conv42 = nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False)
        self.bn42 = nn.BatchNorm2d(width)
        self.relu42 = nn.ReLU(inplace=True)
        
          #convs.append(nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False))
          #bns.append(nn.BatchNorm2d(width))
          #convs.append(nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False))
         
            
            
        #self.convs = nn.ModuleList(convs)
        #self.bns = nn.ModuleList(bns)

        self.conv3 = nn.Conv2d(width*scale, planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes)
        self.relu3 = nn.ReLU(inplace=True)
        
        vgg1 =  models.vgg16(pretrained=False)
        modules1 = list(vgg1.children())[:-2]      # delete the last fc layer.
        self.vgg = nn.Sequential(*modules1)
        
        self.globpool = nn.AvgPool2d(2)
        
        self.fc1 = nn.Linear(512, 30)
        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        
        residual = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        
        spx = torch.split(x, self.width_1, 1)
        
        x1 = self.conv11(spx[0])
        x1 = self.bn11(x1)
        x1 = self.relu11(x1)
        x1 = self.conv12(x1)
        x1 = self.bn12(x1)
        x1 = self.relu12(x1)
        
        x2 = x1 + spx[1]
        x2 = self.conv21(x2)
        x2 = self.bn21(x2)
        x2 = self.relu21(x2)
        x2 = self.conv22(x2)
        x2 = self.bn22(x2)
        x2 = self.relu22(x2)
        
        x3 = x2 + spx[2]
        x3 = self.conv31(x3)
        x3 = self.bn31(x3)
        x3 = self.relu31(x3)
        x3 = self.conv32(x3)
        x3 = self.bn32(x3)
        x3 = self.relu32(x3)
        
        x4 = x3 + spx[3]
        x4 = self.conv41(x4)
        x4 = self.bn41(x4)
        x4 = self.relu41(x4)
        x4 = self.conv22(x4)
        x4 = self.bn42(x4)
        x4 = self.relu42(x4)
        
        
              
        out = torch.cat((x1,x2,x3,x4), 1)
                     
        
        x = self.conv3(out)
        x = self.bn3(x)
        x = self.relu3(x)
        
        x = x - residual
        
        x = self.vgg(x)
        
        x = self.globpool(x)
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        
        x = self.softmax(x)    

        return x

In [8]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0')
model = Res2Net().to(device)

/home/user1/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/user1/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
model = torch.load("Liu_trained_model_merged_final")
model.eval()

results_folder = "Results_Random__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Random__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a=-1

with torch.no_grad():
    for batch_idx, (imgs1, labels1,patch_filename) in enumerate(test_loader_random):
        _, c = torch.max(labels1.data,1)
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org,target = imgs1.to(device,dtype=torch.float), labels1.to(device)
        #img_org = img_org.permute(0, 3, 1, 2)
        
        output = model(img_org)
        

        _, actual = torch.max(target.data, 1)    
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])

Yes_Class 0
0 ('D01_rnd_34_15',) 0 0 0.99
1000 ('D01_rnd_130_89',) 0 0 0.82
2000 ('D01_rnd_110_123',) 0 0 1.0
3000 ('D01_rnd_99_82',) 0 15 0.99
4000 ('D01_rnd_74_78',) 0 0 0.99
Yes_Class 28
5000 ('D29_rnd_135_46',) 28 28 0.5
6000 ('D29_rnd_113_27',) 28 28 1.0
7000 ('D29_rnd_108_101',) 28 28 0.99
8000 ('D29_rnd_113_57',) 28 28 1.0
Yes_Class 22
9000 ('D23_rnd_25_88',) 22 29 0.29
10000 ('D23_rnd_135_21',) 22 22 0.94
11000 ('D23_rnd_138_35',) 22 22 1.0
12000 ('D23_rnd_130_116',) 22 22 0.64
Yes_Class 27
13000 ('D28_rnd_69_108',) 27 5 0.74
14000 ('D28_rnd_99_47',) 27 5 0.54
15000 ('D28_rnd_138_45',) 27 27 0.98
16000 ('D28_rnd_81_0',) 27 27 0.95
Yes_Class 3
17000 ('D04_rnd_240_20',) 3 3 1.0
18000 ('D04_rnd_215_34',) 3 3 1.0
19000 ('D04_rnd_197_12',) 3 3 1.0
20000 ('D04_rnd_8_82',) 3 3 1.0
21000 ('D04_rnd_215_74',) 3 3 1.0
22000 ('D04_rnd_160_23',) 3 3 1.0
23000 ('D04_rnd_240_113',) 3 3 1.0
Yes_Class 14
24000 ('D15_rnd_41_87',) 14 14 1.0
25000 ('D15_rnd_111_74',) 14 14 1.0
26000 ('D15_rnd_101_

In [10]:
csv_dir = img_dir = "Results_Random__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

with open(os.path.join(csv_dir, 'Image_Level_Results_Random.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Patches",\
                         "Total Patches(Correct Classified Images)",\
                         "Correct Predicted Patches(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Softmax Probability of Correct Patch(Only Correct Images)"])

In [11]:
for f in files:
    df = pd.read_csv(f)
    data = df.sort_values(by=['Patch_Filename'])
    classname = Path(f).stem
    classname = int(classname.split("_")[2])
    a="a"

    true_image_class = classname
    total_images_perclass = 0
    correct_images_perclass = 0
    total_patches_perclass = 0

    total_class_votes =0
    total_class_patches =0
    prob_avg_correct_patch = 0.0
    votes = 0

    arr_pred_patches = []
    arr_pred_patches_prob = []
    total_correc_img_patches = 0


    for ind in data.index:
            filename = df['Patch_Filename'][ind]
            pred_patch_class = df['Predicted Class'][ind]
            pred_patch_prob = df['Probability of Predicted Class'][ind]
            
            filename = filename.split("_")
            file_length = len(filename)
            initial_filename = filename[:-1]
            patch_name = filename[file_length-1]
            #print(initial_filename)
            #print(patch_name)

            if(a!= initial_filename and a=="a"):
                a = initial_filename



            if(a!=initial_filename and a!="a"):
                total_images_perclass = total_images_perclass + 1

                counts = np.bincount(arr_pred_patches)
                pred_image_class = np.argmax(counts)
                votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                s=0
                
                if(pred_image_class == true_image_class):
                    correct_images_perclass = correct_images_perclass + 1
                    total_class_votes = total_class_votes + votes
                    total_class_patches = total_class_patches + len(arr_pred_patches)
                    
                    total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                    z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                    for i in range(0,len(arr_pred_patches)):
                        if(z[i]==1):
                            s= s+1
                            prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
            
                arr_pred_patches = []
                arr_pred_patches_prob = []
                a = initial_filename



            if(a==initial_filename):
                total_patches_perclass = total_patches_perclass + 1
                arr_pred_patches.append(pred_patch_class)
                arr_pred_patches_prob.append(pred_patch_prob)



    total_images_perclass = total_images_perclass + 1

    counts = np.bincount(arr_pred_patches)
    pred_image_class = np.argmax(counts)
    votes = np.count_nonzero(arr_pred_patches==pred_image_class)

    if(pred_image_class == true_image_class):
        correct_images_perclass = correct_images_perclass + 1
        total_class_votes = total_class_votes + votes
        total_class_patches = total_class_patches + len(arr_pred_patches)
        
        total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

        z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
        for i in range(0,len(arr_pred_patches)):
            if(z[i]==1):
                prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

    arr_pred_patches = []
    arr_pred_patches_prob = []
    
    if(correct_images_perclass!=0):
        prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
        prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
        avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
        print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

        with open(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
    else:
        with open(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
            

df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'))

ILA_random = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_random}%")

36 4608 33 3363 79.62 0.85
32 4096 32 3739 91.28 0.92
43 5504 42 5080 94.49 0.97
38 4864 34 4020 92.37 0.88
32 4096 31 3568 89.92 0.95
34 4352 34 3890 89.38 0.84
32 4096 30 3773 98.26 0.99
40 5120 38 4779 98.25 0.98
44 5632 43 4906 89.14 0.91
47 6016 46 5463 92.78 0.96
35 4480 29 2935 79.07 0.85
31 3968 31 3562 89.77 0.92
34 4352 33 3679 87.1 0.92
34 4352 32 3606 88.04 0.93
33 4224 31 3772 95.06 0.97
31 3968 30 3102 80.78 0.8
34 4352 32 3334 81.4 0.83
31 3968 26 2223 66.8 0.81
31 3968 30 3641 94.82 0.98
56 7168 49 5659 90.23 0.92
51 6528 49 6181 98.55 0.99
33 4224 32 3531 86.21 0.89
33 4224 33 4118 97.49 0.99
31 3968 18 1752 76.04 0.76
30 3840 29 3269 88.07 0.93
32 4096 32 3538 86.38 0.94
32 4096 32 4013 97.97 0.98
35 4480 34 4097 94.14 0.93
35 4480 33 3817 90.36 0.92
Image Level Accuracy: 94.03846153846153%


In [12]:
model.eval()

results_folder = "Results_Similar__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Similar__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a=-1

with torch.no_grad():
    for batch_idx, (imgs1, labels1,patch_filename) in enumerate(test_loader_similar):
        _, c = torch.max(labels1.data,1)
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org,target = imgs1.to(device,dtype=torch.float), labels1.to(device)
        #img_org = img_org.permute(0, 3, 1, 2)
        
        output = model(img_org)
        

        _, actual = torch.max(target.data, 1)    
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])

Yes_Class 0
0 ('D01_obj_36_79',) 0 0 1.0
1000 ('D01_tex_29_58',) 0 0 1.0
2000 ('D01_nat_45_9',) 0 0 0.84
3000 ('D01_obj_2_7',) 0 0 0.96
4000 ('D01_nat_29_4',) 0 0 0.92
Yes_Class 28
5000 ('D29_nat_1_62',) 28 28 1.0
6000 ('D29_nat_30_58',) 28 28 0.67
7000 ('D29_nat_9_75',) 28 28 0.99
8000 ('D29_nat_2_121',) 28 28 0.98
Yes_Class 22
9000 ('D23_obj_1_37',) 22 22 1.0
10000 ('D23_obj_30_11',) 22 22 1.0
11000 ('D23_tex_2_112',) 22 22 0.63
12000 ('D23_obj_29_126',) 22 22 0.96
Yes_Class 27
13000 ('D28_nat_25_124',) 27 19 0.94
14000 ('D28_nat_45_0',) 27 27 0.7
15000 ('D28_nat_30_56',) 27 27 0.38
16000 ('D28_tex_39_17',) 27 27 0.71
Yes_Class 3
17000 ('D04_tex_7_105',) 3 3 1.0
18000 ('D04_obj_5_113',) 3 3 1.0
19000 ('D04_color_2_61',) 3 3 1.0
20000 ('D04_nat_36_100',) 3 3 1.0
Yes_Class 14
21000 ('D15_nat_2_60',) 14 14 0.91
22000 ('D15_nat_49_90',) 14 14 0.64
23000 ('D15_nat_36_48',) 14 14 1.0
24000 ('D15_obj_29_80',) 14 14 1.0
Yes_Class 20
25000 ('D21_nat_49_96',) 20 20 1.0
26000 ('D21_tex_25_99',)

In [13]:
csv_dir = img_dir = "Results_Similar__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

with open(os.path.join(csv_dir, 'Image_Level_Results_Random.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Patches",\
                         "Total Patches(Correct Classified Images)",\
                         "Correct Predicted Patches(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Softmax Probability of Correct Patch(Only Correct Images)"])

In [14]:
for f in files:
    df = pd.read_csv(f)
    data = df.sort_values(by=['Patch_Filename'])
    classname = Path(f).stem
    classname = int(classname.split("_")[2])
    a="a"

    true_image_class = classname
    total_images_perclass = 0
    correct_images_perclass = 0
    total_patches_perclass = 0

    total_class_votes =0
    total_class_patches =0
    prob_avg_correct_patch = 0.0
    votes = 0

    arr_pred_patches = []
    arr_pred_patches_prob = []
    total_correc_img_patches = 0


    for ind in data.index:
            filename = df['Patch_Filename'][ind]
            pred_patch_class = df['Predicted Class'][ind]
            pred_patch_prob = df['Probability of Predicted Class'][ind]
            
            filename = filename.split("_")
            file_length = len(filename)
            initial_filename = filename[:-1]
            patch_name = filename[file_length-1]
            #print(initial_filename)
            #print(patch_name)

            if(a!= initial_filename and a=="a"):
                a = initial_filename



            if(a!=initial_filename and a!="a"):
                total_images_perclass = total_images_perclass + 1

                counts = np.bincount(arr_pred_patches)
                pred_image_class = np.argmax(counts)
                votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                s=0
                
                if(pred_image_class == true_image_class):
                    correct_images_perclass = correct_images_perclass + 1
                    total_class_votes = total_class_votes + votes
                    total_class_patches = total_class_patches + len(arr_pred_patches)
                    
                    total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                    z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                    for i in range(0,len(arr_pred_patches)):
                        if(z[i]==1):
                            s= s+1
                            prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
            
                arr_pred_patches = []
                arr_pred_patches_prob = []
                a = initial_filename



            if(a==initial_filename):
                total_patches_perclass = total_patches_perclass + 1
                arr_pred_patches.append(pred_patch_class)
                arr_pred_patches_prob.append(pred_patch_prob)



    total_images_perclass = total_images_perclass + 1

    counts = np.bincount(arr_pred_patches)
    pred_image_class = np.argmax(counts)
    votes = np.count_nonzero(arr_pred_patches==pred_image_class)

    if(pred_image_class == true_image_class):
        correct_images_perclass = correct_images_perclass + 1
        total_class_votes = total_class_votes + votes
        total_class_patches = total_class_patches + len(arr_pred_patches)
        
        total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

        z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
        for i in range(0,len(arr_pred_patches)):
            if(z[i]==1):
                prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

    arr_pred_patches = []
    arr_pred_patches_prob = []
    
    if(correct_images_perclass!=0):
        prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
        prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
        avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
        print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

        with open(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
    else:
        with open(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
            

df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'))

ILA_similar = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_similar}%")

32 4096 29 3001 80.85 0.87
32 4096 32 3806 92.92 0.94
32 4096 32 3881 94.75 0.97
32 4096 32 3619 88.35 0.91
32 4096 32 3863 94.31 0.97
32 4096 29 3397 91.51 0.89
32 4096 32 4079 99.58 0.99
32 4096 32 3955 96.56 0.98
32 4096 30 3374 87.86 0.89
32 4096 32 3800 92.77 0.95
32 4096 32 3533 86.25 0.88
32 4096 32 3714 90.67 0.94
32 4096 32 3587 87.57 0.91
32 4096 32 3891 95.0 0.96
32 4096 32 3942 96.24 0.97
32 4096 31 3529 88.94 0.84
32 4096 30 3285 85.55 0.81
32 4096 28 2883 80.44 0.91
32 4096 28 3171 88.48 0.93
32 4096 31 3801 95.79 0.96
32 4096 32 4044 98.73 0.99
32 4096 27 3102 89.76 0.86
32 4096 24 2164 70.44 0.76
32 4096 32 4058 99.07 0.99
32 4096 27 2575 74.51 0.72
32 4096 32 3414 83.35 0.92
32 4096 31 3693 93.07 0.96
32 4096 32 3954 96.53 0.97
32 4096 30 3654 95.16 0.95
32 4096 32 3930 95.95 0.95
Image Level Accuracy: 95.72916666666667%


In [15]:
model.eval()

results_folder = "Results_Merged__"
if os.path.exists(results_folder):
    # If it exists, delete the folder and its content
    print("Deleting existing Results_Merged__ folder...")
    for file in os.listdir(results_folder):
        file_path = os.path.join(results_folder, file)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                os.rmdir(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

os.makedirs(results_folder, exist_ok=True)

a=-1

with torch.no_grad():
    for batch_idx, (imgs1, labels1,patch_filename) in enumerate(test_loader_merged):
        _, c = torch.max(labels1.data,1)
        d = c.cpu().numpy()[0]

        if(d!=a):
            print("Yes_Class",d)
            a= d
            z = d
            file_class = os.path.join(results_folder, f"Test_Class_{z}.csv")

            with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Batch_Id","Patch_Filename","True Class","Predicted Class","Probability of Predicted Class"])
    


        img_org,target = imgs1.to(device,dtype=torch.float), labels1.to(device)
        #img_org = img_org.permute(0, 3, 1, 2)
        
        output = model(img_org)
        

        _, actual = torch.max(target.data, 1)    
        _, predicted = torch.max(output.data, 1)

        y_true = actual.cpu().numpy()[0]
        y_pred =predicted.cpu().numpy()[0]

        prob_y_pred = output[0][y_pred]
        prob_y_pred = prob_y_pred.cpu().numpy()
        prob_y_pred = np.around(prob_y_pred,decimals=2)

        if(batch_idx % 1000 == 0):
          print(batch_idx,patch_filename,y_true,y_pred,prob_y_pred)
        
        
        with open(file_class, 'a+', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([batch_idx,patch_filename,y_true,y_pred,prob_y_pred])

Yes_Class 0
0 ('D01_rnd_34_15',) 0 0 0.99
1000 ('D01_rnd_62_65',) 0 0 1.0
2000 ('D01_obj_25_86',) 0 0 0.9
3000 ('D01_rnd_141_66',) 0 13 0.6
4000 ('D01_rnd_86_8',) 0 0 1.0
5000 ('D01_tex_39_45',) 0 0 1.0
6000 ('D01_nat_2_29',) 0 0 0.66
7000 ('D01_rnd_35_66',) 0 0 0.84
8000 ('D01_tex_25_49',) 0 0 0.96
Yes_Class 28
9000 ('D29_rnd_2_31',) 28 28 1.0
10000 ('D29_rnd_30_83',) 28 5 0.39
11000 ('D29_obj_1_126',) 28 28 1.0
12000 ('D29_rnd_59_47',) 28 28 1.0
13000 ('D29_rnd_7_78',) 28 28 1.0
14000 ('D29_rnd_160_102',) 28 28 0.99
15000 ('D29_nat_39_81',) 28 28 1.0
16000 ('D29_rnd_141_88',) 28 28 0.37
17000 ('D29_rnd_129_79',) 28 28 1.0
Yes_Class 22
18000 ('D23_obj_29_89',) 22 22 0.83
19000 ('D23_rnd_70_60',) 22 10 0.49
20000 ('D23_rnd_55_39',) 22 22 0.84
21000 ('D23_rnd_43_114',) 22 22 0.99
22000 ('D23_nat_9_57',) 22 22 1.0
23000 ('D23_tex_17_41',) 22 22 0.99
24000 ('D23_tex_30_41',) 22 22 1.0
Yes_Class 27
25000 ('D28_color_4_50',) 27 27 0.88
26000 ('D28_rnd_67_107',) 27 6 0.48
27000 ('D28_obj_25_

221000 ('D27_tex_17_48',) 26 26 1.0
222000 ('D27_obj_25_104',) 26 26 0.88
Yes_Class 11
223000 ('D12_nat_45_98',) 11 11 0.96
224000 ('D12_obj_2_105',) 11 11 1.0
225000 ('D12_rnd_130_113',) 11 11 0.99
226000 ('D12_rnd_119_99',) 11 11 0.75
227000 ('D12_tex_30_123',) 11 11 0.63
228000 ('D12_tex_30_113',) 11 11 0.63
229000 ('D12_tex_25_114',) 11 11 1.0
230000 ('D12_rnd_150_82',) 11 11 0.95
231000 ('D12_rnd_71_96',) 11 11 0.98
Yes_Class 15
232000 ('D16_color_4_43',) 15 15 0.92
233000 ('D16_nat_49_9',) 15 0 0.49
234000 ('D16_rnd_73_60',) 15 15 0.98
235000 ('D16_nat_25_56',) 15 15 0.66
236000 ('D16_nat_29_115',) 15 15 0.51
237000 ('D16_rnd_110_76',) 15 15 1.0
238000 ('D16_color_4_97',) 15 15 1.0
239000 ('D16_rnd_35_108',) 15 15 0.88
Yes_Class 24
240000 ('D25_rnd_80_113',) 24 24 0.63
241000 ('D25_rnd_47_85',) 24 24 0.98
242000 ('D25_rnd_117_60',) 24 24 0.92
243000 ('D25_rnd_5_119',) 24 17 0.51
244000 ('D25_rnd_63_19',) 24 24 0.61
245000 ('D25_rnd_117_2',) 24 24 0.81
246000 ('D25_nat_29_90',) 24

In [16]:
csv_dir = img_dir = "Results_Merged__/"
data_path = os.path.join(img_dir,'*csv')
files = glob.glob(data_path)

with open(os.path.join(csv_dir, 'Image_Level_Results_Random.csv'), 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Image Class Label","Number of Images",\
                         "Correct Predicted Images",\
                         "Number of Patches",\
                         "Total Patches(Correct Classified Images)",\
                         "Correct Predicted Patches(Correct Classified Images)",\
                         "Precetange Votes Per Image(Only Correct Images)",\
                         "Average Softmax Probability of Correct Patch(Only Correct Images)"])

In [17]:
for f in files:
    df = pd.read_csv(f)
    data = df.sort_values(by=['Patch_Filename'])
    classname = Path(f).stem
    classname = int(classname.split("_")[2])
    a="a"

    true_image_class = classname
    total_images_perclass = 0
    correct_images_perclass = 0
    total_patches_perclass = 0

    total_class_votes =0
    total_class_patches =0
    prob_avg_correct_patch = 0.0
    votes = 0

    arr_pred_patches = []
    arr_pred_patches_prob = []
    total_correc_img_patches = 0


    for ind in data.index:
            filename = df['Patch_Filename'][ind]
            pred_patch_class = df['Predicted Class'][ind]
            pred_patch_prob = df['Probability of Predicted Class'][ind]
            
            filename = filename.split("_")
            file_length = len(filename)
            initial_filename = filename[:-1]
            patch_name = filename[file_length-1]
            #print(initial_filename)
            #print(patch_name)

            if(a!= initial_filename and a=="a"):
                a = initial_filename



            if(a!=initial_filename and a!="a"):
                total_images_perclass = total_images_perclass + 1

                counts = np.bincount(arr_pred_patches)
                pred_image_class = np.argmax(counts)
                votes = np.count_nonzero(arr_pred_patches==pred_image_class)
                s=0
                
                if(pred_image_class == true_image_class):
                    correct_images_perclass = correct_images_perclass + 1
                    total_class_votes = total_class_votes + votes
                    total_class_patches = total_class_patches + len(arr_pred_patches)
                    
                    total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

                    z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
                    for i in range(0,len(arr_pred_patches)):
                        if(z[i]==1):
                            s= s+1
                            prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]
            
                arr_pred_patches = []
                arr_pred_patches_prob = []
                a = initial_filename



            if(a==initial_filename):
                total_patches_perclass = total_patches_perclass + 1
                arr_pred_patches.append(pred_patch_class)
                arr_pred_patches_prob.append(pred_patch_prob)



    total_images_perclass = total_images_perclass + 1

    counts = np.bincount(arr_pred_patches)
    pred_image_class = np.argmax(counts)
    votes = np.count_nonzero(arr_pred_patches==pred_image_class)

    if(pred_image_class == true_image_class):
        correct_images_perclass = correct_images_perclass + 1
        total_class_votes = total_class_votes + votes
        total_class_patches = total_class_patches + len(arr_pred_patches)
        
        total_correc_img_patches = total_correc_img_patches + len(arr_pred_patches)

        z = np.where(np.array(arr_pred_patches)==true_image_class,1,0)
        for i in range(0,len(arr_pred_patches)):
            if(z[i]==1):
                prob_avg_correct_patch = prob_avg_correct_patch + arr_pred_patches_prob[i]

    arr_pred_patches = []
    arr_pred_patches_prob = []
    
    if(correct_images_perclass!=0):
        prob_avg_correct_patch = prob_avg_correct_patch / total_class_votes
        prob_avg_correct_patch = np.around(prob_avg_correct_patch,decimals=2)
        avg_vote_perclass = np.around((total_class_votes*100) / total_correc_img_patches,decimals=2)
        print(total_images_perclass,total_patches_perclass,correct_images_perclass,total_class_votes,avg_vote_perclass,prob_avg_correct_patch)  

        with open(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
    else:
        with open(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'), 'a+', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([classname,total_images_perclass,correct_images_perclass,total_patches_perclass,\
                             total_correc_img_patches,total_class_votes,\
                             avg_vote_perclass,\
                             prob_avg_correct_patch])
            

df = pd.read_csv(os.path.join(csv_dir + 'Image_Level_Results_Random.csv'))

ILA_merged = sum(df['Correct Predicted Images']) / sum(df['Number of Images']) * 100

print(f"Image Level Accuracy: {ILA_merged}%")

68 8704 62 6364 80.19 0.86
64 8192 64 7545 92.1 0.93
75 9600 74 8961 94.61 0.97
70 8960 66 7639 90.42 0.9
64 8192 63 7431 92.15 0.96
66 8448 63 7287 90.36 0.87
64 8192 62 7852 98.94 0.99
72 9216 70 8734 97.48 0.98
76 9728 73 8280 88.61 0.9
79 10112 78 9263 92.78 0.96
67 8576 61 6468 82.84 0.87
63 8064 63 7276 90.23 0.93
66 8448 65 7266 87.33 0.91
66 8448 64 7497 91.52 0.94
65 8320 63 7714 95.66 0.97
63 8064 61 6631 84.93 0.82
66 8448 62 6619 83.4 0.82
63 8064 54 5106 73.87 0.87
63 8064 58 6812 91.76 0.96
88 11264 80 9460 92.38 0.94
83 10624 81 10225 98.62 0.99
65 8320 59 6633 87.83 0.88
32 4096 24 2164 70.44 0.76
65 8320 65 8176 98.27 0.99
63 8064 45 4327 75.12 0.74
62 7936 61 6683 85.59 0.92
64 8192 63 7231 89.67 0.95
64 8192 64 7967 97.25 0.97
67 8576 64 7751 94.62 0.94
67 8576 65 7747 93.11 0.94
Image Level Accuracy: 94.85%


In [18]:
print(f"Image Level Accuracy (Trained on Merged Set, Tested on Random Set): {ILA_random}%")
print(f"Image Level Accuracy (Trained on Merged Set, Tested on Similar Set): {ILA_similar}%")
print(f"Image Level Accuracy (Trained on Merged Set, Tested on Merged Set): {ILA_merged}%")

Image Level Accuracy (Trained on Merged Set, Tested on Random Set): 94.03846153846153%
Image Level Accuracy (Trained on Merged Set, Tested on Similar Set): 95.72916666666667%
Image Level Accuracy (Trained on Merged Set, Tested on Merged Set): 94.85%
